In [ ]:
# !pip install --upgrade git+https://github.com/scikit-learn-contrib/hdbscan.git
# !pip install --upgrade BERTopic

In [4]:
from bertopic import BERTopic
import os 
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
#from .FileTransformer import read_and_process_file
from .bert_score_eval import get_bertscore
from perpexlity_score import perplexity_score
from ..absa import Absa

ImportError: attempted relative import with no known parent package

In [51]:
file_path = 'response1.txt'
df=read_and_process_file(file_path)
df.head(5)

,Character,Dialogue
0,Prompt,"The free market system, competitive capitalism..."
1,Agent1,The evidence suggests that economic growth and...
2,Agent2,While I understand the left leaning lawyer's c...
3,Agent1,As a lawyer representing a left leaning non pr...
4,Agent2,As a conservative lawyer representing a right ...


In [55]:
absa=Absa(df)
new_df=absa.get_absa()

2023-12-05 01:54:46,860 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


   Topic  Count
0      0     13
1      1      9


In [57]:
new_df.head(20)

,Character,Dialogue,sentiment_scores
0,Prompt,"The free market system, competitive capitalism...","{'market': ('Positive', 0.9310367703437805), '..."
1,Agent1,The evidence suggests that economic growth and...,"{'market': ('Negative', 0.9032272100448608), '..."
2,Agent2,While I understand the left leaning lawyer's c...,"{'market': ('Positive', 0.9724560379981995), '..."
3,Agent1,As a lawyer representing a left leaning non pr...,"{'market': ('Negative', 0.7937985062599182), '..."
4,Agent2,As a conservative lawyer representing a right ...,"{'market': ('Positive', 0.8667799830436707), '..."
5,Agent1,I strongly disagree with my conservative count...,"{'market': ('Positive', 0.6113536953926086), '..."
6,Agent2,I strongly disagree with my liberal counterpar...,"{'market': ('Positive', 0.9414671659469604), '..."
7,Agent1,I understand my conservative counterpart's poi...,"{'market': ('Negative', 0.4450836777687073), '..."
8,Agent2,I respectfully disagree with my counterpart's ...,"{'market': ('Positive', 0.7297154664993286), '..."
9,Agent1,"I understand my counterpart's perspective, but...","{'market': ('Positive', 0.5097795724868774), '..."


In [4]:
convs=df['Conversation'].to_list()
print(convs[1])

>>>> 
The world does not need nuclear weapons. Nuclear weapons are a destructive force that poses an existential threat to humanity. The devastating effects of nuclear weapons are well documented, and their use would result in untold suffering and loss of life. Furthermore, the proliferation of nuclear weapons increases the likelihood of nuclear conflict and undermines international stability.
In addition, the development and maintenance of nuclear weapons is a massive drain on resources that could be better spent on addressing pressing global issues such as poverty, inequality, and climate change. The pursuit of nuclear weapons is a misguided and dangerous obsession that serves only to perpetuate a cycle of fear and mistrust among nations.
In conclusion, the world cannot afford to continue down the path of nuclear proliferation. It is time for nations to come together and work towards a world free of nuclear weapons."
Please provide the next point of view.
Agent2 ---


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english")

In [6]:
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = model_embedding.encode(convs)

In [7]:
model = BERTopic(
    n_gram_range=(1, 2),
    vectorizer_model=vectorizer_model,
    nr_topics='auto',
    min_topic_size=5,
    top_n_words=5,
    calculate_probabilities=True).fit(convs, corpus_embeddings)

In [8]:
topics, probabilities = model.transform(convs, corpus_embeddings)

In [9]:
df_topic_freq = model.get_topic_freq()
topics_count = len(df_topic_freq) - 1
df_topic_freq

,Topic,Count
0,-1,31
1,0,27
4,1,18
2,2,17
3,3,8


In [10]:
model.visualize_barchart(top_n_topics=topics_count)

In [11]:
model.get_topic_info()['Representation'][0]

['nuclear', 'weapons', 'world', 'existence', 'aggressive']

In [12]:
new_topics = model.reduce_outliers(convs, topics, strategy="c-tf-idf")

In [13]:
model.update_topics(convs, topics=new_topics, vectorizer_model=vectorizer_model)

2023-12-05 01:02:15,067 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [14]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,44,0_weapons_nuclear_existence_stability,"[weapons, nuclear, existence, stability, natio...",[>>>> \nI understand my counterpart's point th...
1,1,28,1_nuclear_weapons_security_global,"[nuclear, weapons, security, global, nations, ...",[>>>> I understand my liberal counterpart's p...
2,2,20,2_nuclear_deterrence_conflict_security,"[nuclear, deterrence, conflict, security, worl...",[>>>> My colleague's argument that nuclear de...
3,3,9,3_nuclear_nonstate_economic_actors,"[nuclear, nonstate, economic, actors, weapons,...",[>>>> \nWhile I understand my colleague's argu...


In [15]:
text_data=model.get_document_info(convs)
text_data.head(5)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Agent1 ---,1,1_nuclear_weapons_security_global,"[nuclear, weapons, security, global, nations, ...",[>>>> I understand my liberal counterpart's p...,nuclear - weapons - security - global - nation...,0.360528,False
1,>>>> \nThe world does not need nuclear weapons...,0,0_weapons_nuclear_existence_stability,"[weapons, nuclear, existence, stability, natio...",[>>>> \nI understand my counterpart's point th...,weapons - nuclear - existence - stability - na...,0.498553,False
2,>>>> \nThe world needs nuclear weapons to main...,0,0_weapons_nuclear_existence_stability,"[weapons, nuclear, existence, stability, natio...",[>>>> \nI understand my counterpart's point th...,weapons - nuclear - existence - stability - na...,0.388126,False
3,>>>> \nWhile I understand my colleague's argum...,0,0_weapons_nuclear_existence_stability,"[weapons, nuclear, existence, stability, natio...",[>>>> \nI understand my counterpart's point th...,weapons - nuclear - existence - stability - na...,0.277919,False
4,>>>> \nI strongly disagree with my colleague's...,2,2_nuclear_deterrence_conflict_security,"[nuclear, deterrence, conflict, security, worl...",[>>>> My colleague's argument that nuclear de...,nuclear - deterrence - conflict - security - w...,0.497617,False


In [16]:
model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
absa_model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [17]:
def get_aspect_scores(text,aspect_list):
    sentiment_aspect = {}
    for aspect in aspect_list:
        inputs = tokenizer(text, aspect, return_tensors="pt")

        with torch.inference_mode():
            outputs = absa_model(**inputs)

        scores = F.softmax(outputs.logits[0], dim=-1)
        label_id = torch.argmax(scores).item()
        sentiment_aspect[aspect] = (absa_model.config.id2label[label_id], scores[label_id].item())

    print(sentiment_aspect)
    return sentiment_aspect



In [18]:
text_data["Representation"][1]

['weapons',
 'nuclear',
 'existence',
 'stability',
 'nations',
 'led',
 'world',
 'aggressive',
 'deterrent',
 'disagree']

In [19]:
get_aspect_scores(text_data["Document"][1],text_data["Representation"][1])

{'weapons': ('Negative', 0.8678022027015686), 'nuclear': ('Negative', 0.8668756484985352), 'existence': ('Negative', 0.7782708406448364), 'stability': ('Negative', 0.9457460641860962), 'nations': ('Negative', 0.7207546830177307), 'led': ('Negative', 0.7205950021743774), 'world': ('Negative', 0.7314151525497437), 'aggressive': ('Negative', 0.7485793828964233), 'deterrent': ('Negative', 0.8038850426673889), 'disagree': ('Negative', 0.8213804364204407)}


{'weapons': ('Negative', 0.8678022027015686),
 'nuclear': ('Negative', 0.8668756484985352),
 'existence': ('Negative', 0.7782708406448364),
 'stability': ('Negative', 0.9457460641860962),
 'nations': ('Negative', 0.7207546830177307),
 'led': ('Negative', 0.7205950021743774),
 'world': ('Negative', 0.7314151525497437),
 'aggressive': ('Negative', 0.7485793828964233),
 'deterrent': ('Negative', 0.8038850426673889),
 'disagree': ('Negative', 0.8213804364204407)}

In [20]:
for i in range (len(text_data)):
    get_aspect_scores(text_data["Document"][i],text_data["Representation"][i])

{'nuclear': ('Neutral', 0.9567712545394897), 'weapons': ('Neutral', 0.9705731272697449), 'security': ('Neutral', 0.9672309160232544), 'global': ('Neutral', 0.8518043756484985), 'nations': ('Neutral', 0.9869593977928162), 'world': ('Neutral', 0.6551769375801086), 'deterrent': ('Neutral', 0.9334036707878113), 'necessary': ('Neutral', 0.8231979012489319), 'threat': ('Neutral', 0.9535980224609375), 'disagree': ('Negative', 0.8142071962356567)}
{'weapons': ('Negative', 0.8678022027015686), 'nuclear': ('Negative', 0.8668756484985352), 'existence': ('Negative', 0.7782708406448364), 'stability': ('Negative', 0.9457460641860962), 'nations': ('Negative', 0.7207546830177307), 'led': ('Negative', 0.7205950021743774), 'world': ('Negative', 0.7314151525497437), 'aggressive': ('Negative', 0.7485793828964233), 'deterrent': ('Negative', 0.8038850426673889), 'disagree': ('Negative', 0.8213804364204407)}
{'weapons': ('Positive', 0.5935032963752747), 'nuclear': ('Positive', 0.6163008809089661), 'existence